# Making firmware with hls4ml

This is the fun part! Now we'll convert our Keras model into highly parallel FPGA firmware with hls4ml, using only a few lines of code. This is the basic flow:

<img src="images/hls4ml_conifer.png" alt="hls4ml" width="1000" img align="center"/>

So what hlsm4l does is that it takes your TF/Keras/ONNX/Pytorch model and converts it into C++ code that a high-level-synthesis (HLS) tool can read. It automatically adds pipelining stages to your hardware depending on the target FPGA  and clock period constraints (usually at LHC we're on a 2 or 5 nanosecond clock).

After converting your model with hls, you'll see that a lot of C++ code gets generated, which is then passed to Vivado HLS for compilation.

One thing that will become important is the *reuse factor*. The ReuseFactor is our mechanism for tuning the parallelism.

<img src="images/reuse.png" alt="reuse" width="600" img align="center"/>

Since the FPGA on the Pynq board is very tiny, we need to use the multipliers several time so we put a really high reuse factor to make it fit the board 

================================================================

Available resources on the Xilinx Zynq XC7Z020 SoC (FPGA part number \texttt{xc7z020clg400-1}) on the TUL PYNQ-Z2 development board:
```
+-----------------+---------+-------+--------+-------+-----+
|       Name      | BRAM_18K| DSP48E|   FF   |  LUT  | URAM|
+-----------------+---------+-------+--------+-------+-----+
|Available        |      280|    220|  106400|  53200|    0|
+-----------------+---------+-------+--------+-------+-----+
```
(Just considering number of multiplications, our model would use $57*32+32*16+16*3+3*16+16*32+32*57= 4,768$ multiplications and we only have 220 DSPs!)


Let's make bitfiles both of the large and the compressed model,setting the reuse factor to 64:

In [ ]:
import hls4ml
import util
import h5py
import numpy as np
import tensorflow as tf

from qkeras.utils import _add_supported_quantized_objects
co = {}; _add_supported_quantized_objects(co)

hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = []  
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

with h5py.File('Ato4l_dataset.h5', 'r') as file:
    signal_test_data = np.array(file['Data'])

# First the baseline:
autoencoder = tf.keras.models.load_model('baseline_ae.h5')

# Then the compressed model:
q_autoencoder = tf.keras.models.load_model('qkeras_ae.h5', custom_objects=co)

In [ ]:
# Lets convert the baseline model into a bitfile for PYNQ
config = hls4ml.utils.config_from_keras_model(autoencoder, granularity='name')
config['Model']['Strategy'] = 'Resource'

for layer in config['LayerName'].keys():
    config['LayerName'][layer]['ReuseFactor'] = 64 #Use the same resources multiple times. This is neccessary in this case because the FPGA is small.
hls_model = hls4ml.converters.convert_from_keras_model(autoencoder,
                                                         hls_config=config,
                                                         backend='VivadoAccelerator', #You need this backend to generate firmware for Zynq
                                                         output_dir='baseline_ae_pynq',
                                                         board='pynq-z2') # This is our FPGA!
                                                   
hls_model.compile()

y_hls4ml = hls_model.predict(np.ascontiguousarray(signal_test_data))
hls_model.build(csim=False, synth=True, export=True)
hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(hls_model)

# Package the model and some test data to be moved over to the Pynq
util.package(hls_model, signal_test_data, y_hls4ml,name="pynq_pack_baseline")



In [ ]:
# Lets convert the QKeras model into a bitfile for PYNQ
config = hls4ml.utils.config_from_keras_model(q_autoencoder, granularity='name')
config['Model']['Strategy'] = 'Resource'
for layer in config['LayerName'].keys():
    config['LayerName'][layer]['ReuseFactor'] = 64
q_hls_model = hls4ml.converters.convert_from_keras_model(q_autoencoder,
                                                         hls_config=config,
                                                         backend='VivadoAccelerator',
                                                         output_dir='qkeras_ae_pynq',
                                                         board='pynq-z2')                                                
q_hls_model.compile()

y_q_hls4ml = q_hls_model.predict(np.ascontiguousarray(signal_test_data))
q_hls_model.build(csim=False, synth=True, export=True)
hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(q_hls_model)
util.package(q_hls_model, signal_test_data, y_q_hls4ml,name="pynq_pack_qkeras)

We made it and have our bit file! From the synthesis reports we can check how many resoures each network is consuming as well as the total latency.
We can get the latency from the C synthesis report in `qkeras_ae/myproject_prj/solution1/syn/report/myproject_csynth.rpt` and the resource consumption from `qkeras_ae/util.rpt`:

In [ ]:
from pathlib import Path

def getReports(indir):
    
    data_ = {}
    
    report_vsynth = Path('{}/vivado_synth.rpt'.format(indir))
    report_csynth = Path('{}/myproject_prj/solution1/syn/report/myproject_csynth.rpt'.format(indir))
    report_pynq = Path('{}/util.rpt'.format(indir))

    if report_csynth.is_file():    
        with report_csynth.open() as report:
          lines = np.array(report.readlines())
          lat_line = lines[np.argwhere(np.array(['Latency (cycles)' in line for line in lines])).flatten()[0] + 3]
          #data_['latency_clks'] = round(int(lat_line.split('|')[2]))
          data_['Latency']   = lat_line.split('|')[4]
          data_['Initiation Interval']   = round(int(lat_line.split('|')[6]))
    

    if report_vsynth.is_file():
        # Get the resources from the logic synthesis report 
        with report_vsynth.open() as report:
          lines = np.array(report.readlines())
          lut   = int(lines[np.array(['CLB LUTs*' in line for line in lines])][0].split('|')[2])
          ff    = int(lines[np.array(['CLB Registers' in line for line in lines])][0].split('|')[2])
          bram  = float(lines[np.array(['Block RAM Tile' in line for line in lines])][0].split('|')[2])
          dsp   = int(lines[np.array(['DSPs' in line for line in lines])][0].split('|')[2])
          lut_rel = round(float(lines[np.array(['CLB LUTs*' in line for line in lines])][0].split('|')[5]),1)
          ff_rel  = round(float(lines[np.array(['CLB Registers' in line for line in lines])][0].split('|')[5]),1)
          bram_rel= round(float(lines[np.array(['Block RAM Tile' in line for line in lines])][0].split('|')[5]),1)
          dsp_rel = round(float(lines[np.array(['DSPs' in line for line in lines])][0].split('|')[5]),1)
        
          data_['LUTs']     = "{} ({}%)".format(lut  ,lut_rel )
          data_['FFs']      = "{} ({}%)".format(ff   ,ff_rel  )
          data_['BRAMs']    = "{} ({}%)".format(bram ,bram_rel)
          data_['DSPs']     = "{} ({}%)".format(dsp  ,dsp_rel )    
          
    else:        
        # Get the resources from the logic synthesis report 
        with report_pynq.open() as report:
          lines = np.array(report.readlines())
          top_line = lines[np.argwhere(np.array(['(top)' in line for line in lines])).flatten()[0]]
          data_['Total LUTs']   = (top_line.split('|')[3])
          data_['Logic LUTs']   = (top_line.split('|')[4])
          data_['LUTRAMs']   = (top_line.split('|')[5])
          data_['SRLs']   = (top_line.split('|')[6])
          data_['FFs']   = (top_line.split('|')[7])
          data_['RAMB36']   = (top_line.split('|')[8])
          data_['RAMB18']   = (top_line.split('|')[9])
          data_['DSP Blocks']   = (top_line.split('|')[10])
    
    return data_

data_baseline = getReports('baseline_ae_pynq')
data_qkeras = getReports('qkeras_ae_pynq')

print("\n ON PYNQ")
print("\n BASELINE MODEL \n")
for key in data_baseline.keys():
    print("{} = {}".format(key,data_baseline[key]))
print("\n QKERAS MODEL \n")
for key in data_qkeras.keys():
    print("{} = {}".format(key,data_qkeras[key]))   

So we see that, despite having the same latency, the resource constumption for the compressed model is significantly smaller! That in turn means that we don't need to reuse the same multipliers as many times as we have defined (by setting reuse to 64). So quantization can improve the latency, in terms of allowing us to use a *lower reuse factor*.

## Compare to Level-1 trigger FPGA

Deployment on the Pynq is of course a toy study and the FPGAs we have in the Level-1 trigger are much much bigger (and much much more expensive). For fun, let's see what the latency would be with a fully parallel implementation (reuse factor of 1) on a Xilinx VU9P FPGA.

Available resources on the Xilinx Virtex Ultrascale 9+ FPGA:
```
+---------------------+---------+-------+---------+---------+-----+
|         Name        | BRAM_18K| DSP48E|    FF   |   LUT   | URAM|
+---------------------+---------+-------+---------+---------+-----+
|Available            |     4320|   6840|  2364480|  1182240|  960|
+---------------------+---------+-------+---------+---------+-----+
```
Thirty times more DSPs available than on the Pynq!

In [ ]:
# Baseline model
config = hls4ml.utils.config_from_keras_model(autoencoder, granularity='name')
config['Model']['Strategy'] = 'Latency'

for layer in config['LayerName'].keys():
    config['LayerName'][layer]['ReuseFactor'] = 1 #Use the same resources multiple times. This is neccessary in this case because the FPGA is small.
hls_model = hls4ml.converters.convert_from_keras_model(autoencoder,
                                                         hls_config=config,
                                                         output_dir='baseline_ae_vu9p',
                                                         part='xcvu9p-flgb2104-2l-e') # L1T FPGA!
                                                   
hls_model.compile()
hls_model.build(csim=False, synth=True, vsynth=True)

# Compressed model
config = hls4ml.utils.config_from_keras_model(q_autoencoder, granularity='name')
config['Model']['Strategy'] = 'Latency'

for layer in config['LayerName'].keys():
    config['LayerName'][layer]['ReuseFactor'] = 1 #Use the same resources multiple times. This is neccessary in this case because the FPGA is small.
q_hls_model = hls4ml.converters.convert_from_keras_model(q_autoencoder,
                                                         hls_config=config,
                                                         output_dir='qkeras_ae_vu9p',
                                                         part='xcvu9p-flgb2104-2l-e') # L1T FPGA!

q_hls_model.compile()
q_hls_model.build(csim=False, synth=True, vsynth=True)


The latency can still be found in `qkeras_ae_vu9p/myproject_prj/solution1/syn/report/myproject_csynth.rpt`, but the resources are now listed in `qkeras_ae_vu9p/vivado_synth.rpt':

In [ ]:
data_baseline = getReports('baseline_ae_vu9p')
data_qkeras = getReports('qkeras_ae_vu9p')

print("\n ON VU9+")
print("\n BASELINE MODEL \n")
for key in data_baseline.keys():
    print("{} = {}".format(key,data_baseline[key]))
print("\n QKERAS MODEL \n")
for key in data_qkeras.keys():
    print("{} = {}".format(key,data_qkeras[key]))

So we can see that with a completely parallel implementation, the algorithm runs sigificantly faster and would be within the L1 requirements to run in the Global trigger! And since no one would let you deploy an algorithm that uses almost the full board, quantization and pruning are KEY tools for edge ML!